In [ ]:
import os
import cv2
import json
from ultralytics import YOLO
from tqdm import tqdm
from torch.nn.functional import softmax
from scipy.spatial import distance

**### 올바른 json을 input으로 넣었는지 Visualization 수행**

In [3]:
# JSON 및 비디오 경로 설정
json_dir = r"D:\Falldown\Dataset\Video_Dataset\Json_combined\Test"
original_video_dir = r"D:\Falldown\Dataset\Video_Dataset\Video\Test"
output_dir = r"D:\Falldown\code-git\Visualized_Video_Json"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 파일명 리스트
filenames = [
    "00028_H_A_FY_C1",
    "00130_H_A_FY_C2",
    "00175_H_A_BY_C3",
    "00712_H_D_BY_C4",
    "01757_Y_E_SY_C5",
    "01931_Y_A_SY_C6",
    "01224_O_E_N_C7",
    "01251_O_E_N_C8"
]

# 시각화를 위한 함수
def visualize_results(frame, bbox, results_pose, label_text, target_size, original_width, original_height):
    if not bbox and not results_pose: # 데이터가 없으면 시각화 작업 없이 바로 반환
        return frame
    
    color_map = {
        'Normal': (0, 255, 0),      # Normal: 초록색
        'Danger': (0, 165, 255),    # Danger: 주황색
        'Fall': (0, 0, 255)         # Fall: 빨간색
    }
    color = color_map.get(label_text, (255, 255, 255))  # 기본값은 흰색

    h, w = frame.shape[:2]
    scale = min(target_size[0] / w, target_size[1] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    pad_w, pad_h = (target_size[0] - new_w) // 2, (target_size[1] - new_h) // 2

    # Yolo bbox 시각화
    for adjusted_bbox in bbox:
        x1, y1, x2, y2 = adjusted_bbox
        # 640x640으로 resize와 padding 된 yolo 좌표를 원본 동영상 좌표로 복구
        original_x1 = (x1 - pad_w) * (original_width / new_w)
        original_y1 = (y1 - pad_h) * (original_height / new_h)
        original_x2 = (x2 - pad_w) * (original_width / new_w)
        original_y2 = (y2 - pad_h) * (original_height / new_h)

        label = f"Class: {label_text}"
        cv2.rectangle(frame, (int(original_x1), int(original_y1)), (int(original_x2), int(original_y2)), color, thickness=3)
        cv2.putText(frame, label, (int(original_x1), int(original_y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, thickness=2)

    # Mediapipe 관절 좌표 시각화
    for landmark in results_pose:
        global_x = landmark[0]
        global_y = landmark[1]
        # 640x640으로 resize와 padding 된 landmark 좌표를 원본 동영상 좌표로 복구
        original_landmark_x = (global_x - pad_w) * (original_width / new_w)
        original_landmark_y = (global_y - pad_h) * (original_height / new_h)

        cv2.circle(frame, (int(original_landmark_x), int(original_landmark_y)), radius=3, color=(0, 255, 255), thickness=-1)

    return frame

# JSON 파일과 비디오 처리
for filename in tqdm(filenames, desc="Processing files"):
    json_path = os.path.join(json_dir, filename + ".json")
    video_path = os.path.join(original_video_dir, filename + ".mp4")
    output_path = os.path.join(output_dir, filename + "_visualized.mp4")
    target_size = (640, 640)

    # JSON 로드
    with open(json_path, "r") as f:
        json_data = json.load(f)

    # 비디오 로드
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 비디오 저장 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (original_width, original_height))

    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 6 프레임마다 처리
        frame_key = f"frame_{frame_idx}"
        if frame_idx % 6 == 0 and frame_key in json_data["pose_data"]:
            frame_data = json_data["pose_data"][frame_key]

            # bbox 데이터 가져오기
            bbox_data = frame_data.get("bbox", {})
            bbox = []
            if isinstance(bbox_data, dict):  # bbox_data가 딕셔너리일 때만 처리
                bbox = [[
                    bbox_data.get("x1", 0),
                    bbox_data.get("y1", 0),
                    bbox_data.get("x2", 0),
                    bbox_data.get("y2", 0)
                ]]

            # landmarks 데이터 가져오기
            results_pose = [
                (landmark_data["x"] * target_size[0], landmark_data["y"] * target_size[1])
                for key, landmark_data in frame_data.items()
                if key.startswith("landmark_") and "x" in landmark_data and "y" in landmark_data
            ]
            
            label_text = frame_data.get("class", "Unknown") # class 데이터 가져오기

            # 시각화 수행
            frame = visualize_results(frame, bbox, results_pose, label_text, target_size,
                                      original_width, original_height)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Processed {filename}, output saved to {output_path}")

Processing files:  12%|█▎        | 1/8 [00:23<02:47, 23.90s/it]

Processed 00028_H_A_FY_C1, output saved to D:\Falldown\code-git\Visualized_Video_Json\00028_H_A_FY_C1_visualized.mp4


Processing files:  25%|██▌       | 2/8 [00:48<02:25, 24.32s/it]

Processed 00130_H_A_FY_C2, output saved to D:\Falldown\code-git\Visualized_Video_Json\00130_H_A_FY_C2_visualized.mp4


Processing files:  38%|███▊      | 3/8 [00:49<01:07, 13.55s/it]

...

Processed 01931_Y_A_SY_C6, output saved to D:\Falldown\code-git\Visualized_Video_Json\01931_Y_A_SY_C6_visualized.mp4


Processing files:  88%|████████▊ | 7/8 [01:56<00:13, 13.53s/it]

Processed 01224_O_E_N_C7, output saved to D:\Falldown\code-git\Visualized_Video_Json\01224_O_E_N_C7_visualized.mp4


Processing files: 100%|██████████| 8/8 [01:56<00:00, 14.56s/it]

Processed 01251_O_E_N_C8, output saved to D:\Falldown\code-git\Visualized_Video_Json\01251_O_E_N_C8_visualized.mp4
